## ASHRAE 140 2023

Section 6: Weather Test

In [21]:
# Add to environment variables PYTHONPATH = C:\Users\JFC_WIN\Documents\OpenSimula\src
import OpenSimula as osm
import pandas as pd

case_WD = {
    "name": "case_WD",
    "time_step": 3600,
    "n_time_steps": 8760,
    "initial_time": "01/01/2001 00:00:00",
    "components": [
        # MET_FILE
        {
            "type": "File_met",
            "name": "TMY3",
            "file_type": "TMY3",
            "file_name": "WD100.tmy3"
        },
        # CONSTRUCTION
        {
            "type": "Material",
            "name": "Madera",
            "conductivity": 0.14,
            "density": 530,
            "specific_heat": 900
        },
        {
            "type": "Construction",
            "name": "Pared",
            "solar_alpha": [1, 1],
            "materials": ["Madera"],
            "thicknesses": [0.10],
        },
        # BUILDING
        {
            "type": "Building",
            "name": "WT_Building",
            "file_met": "TMY3",
            "albedo": 0.0
        },
        {
            "type": "Space_type",
            "name": "ashrae_space",
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "4.1667",
            "other_gains_radiant_fraction": 0.6,
            "infiltration": "1"
        },
        {
            "type": "Space",
            "name": "space",
            "building": "WT_Building",
            "space_type": "ashrae_space",
            "floor_area": 48,
            "volume": 48*2.7,
        },
        # Surfaces
        {
            "type": "Exterior_surface",
            "name": "north_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [8,6,0],
            "width": 8,
            "height": 2.7,
            "azimuth": 180,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "west_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [0,6,0],
            "width": 4,
            "height": 2.7,
            "azimuth": -90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_west_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [0,2,0],
            "width": 2.828,
            "height": 2.7,
            "azimuth": -45,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [2,0,0],
            "width": 4,
            "height": 2.7,
            "azimuth": 0,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_east_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [6,0,0],
            "width": 2.828,
            "height": 2.7,
            "azimuth": 45,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "east_wall",
            "construction": "Pared",
            "space": "space",
            "ref_point": [8,2,0],
            "width": 4,
            "height": 2.7,
            "azimuth": 90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "roof",
            "construction": "Pared",
            "space": "space",
            "ref_point": [2,2,3.85],
            "width": 4,
            "height": 4,
            "azimuth": 0,
            "altitude": 90
        },
        {
            "type": "Exterior_surface",
            "name": "west_roof",
            "construction": "Pared",
            "space": "space",
            "ref_point": [0,6,2.7],
            "width": 4,
            "height": 2.307,
            "azimuth": -90,
            "altitude": 60
        },
        {
            "type": "Exterior_surface",
            "name": "east_roof",
            "construction": "Pared",
            "space": "space",
            "ref_point": [8,2,2.7],
            "width": 4,
            "height": 2.307,
            "azimuth": 90,
            "altitude": 60
        },
        {
            "type": "Exterior_surface",
            "name": "south_roof",
            "construction": "Pared",
            "space": "space",
            "ref_point": [2,0,2.7],
            "width": 4,
            "height": 2.307,
            "azimuth": 0,
            "altitude": 60
        },
    ],
}

In [22]:
sim = osm.Simulation()
pro = osm.Project("pro", sim)
pro.read_dict(case_WD)

Reading project data from dictonary
Reading completed.
Checking project: case_WD
ok


In [23]:
pro.component("WT_Building").draw_pyvista()

Widget(value='<iframe src="http://localhost:59836/index.html?ui=P_0x30f18e350_4&reconnect=auto" class="pyvista…

In [24]:
pro.simulate()

Simulating case_WD: 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  End


In [25]:
df_met = pro.component("TMY3").variable_dataframe()
df_graph = pd.DataFrame()
df_graph["date"] = df_met["date"]
df_graph["T"] = df_met["temperature [°C]"]
df_graph["RH"] = df_met["rel_humidity [%]"]
df_graph["T_DP"] = df_met["dew_point_temp [°C]"]
df_graph["W"] = df_met["abs_humidity [g/kg]"]/1000
df_graph["T_WB"] = df_met["wet_bulb_temp [°C]"]
df_graph["wind_speed"] = df_met["wind_speed [m/s]"]
df_graph["wind_direction"] = df_met["wind_direction [°]"]
df_graph["p"] = df_met["pressure [Pa]"]/100
df_graph["total_cloud_cover"] = df_met["total_cloud_cover [%]"]/10
df_graph["opaque_cloud_cover"] = df_met["opaque_cloud_cover [%]"]/10
df_graph["T_sky"] = df_met["sky_temperature [°C]"]
df_graph["h_solar"] = df_met["sol_hour [h]"]
df_graph["hor_tot"] = pro.component("roof").variable("E_dir0").values +  pro.component("roof").variable("E_dif0").values
df_graph["hor_beam"] = pro.component("roof").variable("E_dir0").values 
df_graph["hor_dif"] = pro.component("roof").variable("E_dif0").values
df_graph["south_tot"] = pro.component("south_wall").variable("E_dir0").values +  pro.component("south_wall").variable("E_dif0").values
df_graph["south_beam"] = pro.component("south_wall").variable("E_dir0").values 
df_graph["south_dif"] = pro.component("south_wall").variable("E_dif0").values
df_graph["east_tot"] = pro.component("east_wall").variable("E_dir0").values +  pro.component("east_wall").variable("E_dif0").values
df_graph["east_beam"] = pro.component("east_wall").variable("E_dir0").values 
df_graph["east_dif"] = pro.component("east_wall").variable("E_dif0").values
df_graph["north_tot"] = pro.component("north_wall").variable("E_dir0").values +  pro.component("north_wall").variable("E_dif0").values
df_graph["north_beam"] = pro.component("north_wall").variable("E_dir0").values 
df_graph["north_dif"] = pro.component("north_wall").variable("E_dif0").values
df_graph["west_tot"] = pro.component("west_wall").variable("E_dir0").values +  pro.component("west_wall").variable("E_dif0").values
df_graph["west_beam"] = pro.component("west_wall").variable("E_dir0").values 
df_graph["west_dif"] = pro.component("west_wall").variable("E_dif0").values
df_graph["south_east_tot"] = pro.component("south_east_wall").variable("E_dir0").values +  pro.component("south_east_wall").variable("E_dif0").values
df_graph["south_east_beam"] = pro.component("south_east_wall").variable("E_dir0").values 
df_graph["south_east_dif"] = pro.component("south_east_wall").variable("E_dif0").values
df_graph["south_west_tot"] = pro.component("south_west_wall").variable("E_dir0").values +  pro.component("south_west_wall").variable("E_dif0").values
df_graph["south_west_beam"] = pro.component("south_west_wall").variable("E_dir0").values 
df_graph["south_west_dif"] = pro.component("south_west_wall").variable("E_dif0").values
df_graph["east_roof_tot"] = pro.component("east_roof").variable("E_dir0").values +  pro.component("east_roof").variable("E_dif0").values
df_graph["east_roof_beam"] = pro.component("east_roof").variable("E_dir0").values 
df_graph["east_roof_dif"] = pro.component("east_roof").variable("E_dif0").values
df_graph["south_roof_tot"] = pro.component("south_roof").variable("E_dir0").values +  pro.component("south_roof").variable("E_dif0").values
df_graph["south_roof_beam"] = pro.component("south_roof").variable("E_dir0").values 
df_graph["south_roof_dif"] = pro.component("south_roof").variable("E_dif0").values
df_graph["west_roof_tot"] = pro.component("west_roof").variable("E_dir0").values +  pro.component("west_roof").variable("E_dif0").values
df_graph["west_roof_beam"] = pro.component("west_roof").variable("E_dir0").values 
df_graph["west_roof_dif"] = pro.component("west_roof").variable("E_dif0").values



df_graph.to_excel("WD100_OpenSimula.xlsx")